In [15]:
import requests
import pandas as pd
import numpy as np

from sqlalchemy import create_engine, types, table

oracle_connection_string = 'oracle+cx_oracle://{username}:{password}@{hostname}:{port}/{database}'

engine = create_engine(
    oracle_connection_string.format(
        username='TEMEC',
        password='dA1809Bj',
        hostname='10.100.30.16',
        port='1521',
        database='SINEC1',
    )
)

In [2]:
countries = {'-': 'U.S., total'}
url = 'https://www.census.gov/foreign-trade/schedules/c/country.txt'
r = requests.get(url)
#r = r.json()
data = r.text
for i, line in enumerate(data.split('\n')):
    if i > 5 and i < 245:
        words = line.split('|')
        #print(words)
        #print(f'{i} {words[0].strip()} {words[1].strip()}')
        countries[words[0].strip()] = words[1].strip()
        
countries

{'-': 'U.S., total',
 '1010': 'Greenland',
 '1220': 'Canada',
 '1610': 'Saint Pierre and Miquelon',
 '2010': 'Mexico',
 '2050': 'Guatemala',
 '2080': 'Belize',
 '2110': 'El Salvador',
 '2150': 'Honduras',
 '2190': 'Nicaragua',
 '2230': 'Costa Rica',
 '2250': 'Panama',
 '2320': 'Bermuda',
 '2360': 'Bahamas',
 '2390': 'Cuba',
 '2410': 'Jamaica',
 '2430': 'Turks and Caicos Islands',
 '2440': 'Cayman Islands',
 '2450': 'Haiti',
 '2470': 'Dominican Republic',
 '2481': 'Anguilla',
 '2482': 'British Virgin Islands',
 '2483': 'Saint Kitts and Nevis',
 '2484': 'Antigua and Barbuda',
 '2485': 'Montserrat',
 '2486': 'Dominica',
 '2487': 'Saint Lucia',
 '2488': 'Saint Vincent and the Grenadines',
 '2489': 'Grenada',
 '2720': 'Barbados',
 '2740': 'Trinidad and Tobago',
 '2774': 'Sint Maarten',
 '2777': 'Curacao',
 '2779': 'Aruba',
 '2831': 'Guadeloupe',
 '2839': 'Martinique',
 '3010': 'Colombia',
 '3070': 'Venezuela',
 '3120': 'Guyana',
 '3150': 'Suriname',
 '3170': 'French Guiana',
 '3310': 'Ecuad

## Imports by Country

In [17]:
def update_census(comm_type, comm_lvl, from_date):
    
    tables = {'imp' : {'HS2': 'census_country_imp_h2', 'HS4':'census_country_imp_h4', 'HS6':'census_country_imp'}, 
    'exp' : {'HS2': 'census_country_exp_h2', 'HS4':'census_country_exp_h4', 'HS6':'census_country_exp_h6'}}
    
    columns = {'imp': 'CTY_NAME,I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR',
        'exp': 'CTY_NAME,E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR'}
    
    base = 'https://api.census.gov/data/timeseries/intltrade/'
    flow = 'imports/hs' if comm_type == 'imp' else 'exports/hs'
    param = columns[comm_type]
    
    for country in list(countries.keys()):
        print(country)
        url = f'{base}{flow}?get={param}&COMM_LVL={comm_lvl}&CTY_CODE={country}&time={from_date}&key=9e7627074942764e20dcf7c53a39d8aa7a80f7b1'
        r = requests.get(url)
        code = r.status_code
        if code != 200:
            print('Not able to get data {}'.format(code))
            continue
        r = r.json()
        df_state = pd.DataFrame(r[1:], columns=r[0])
        dtyp = {c:types.VARCHAR(df_state[c].str.len().max())
            for c in df_state.columns[df_state.dtypes == 'object'].tolist()}
        df_state.to_sql(con=engine, name=tables[comm_type][comm_lvl], if_exists='append', index=False, dtype=dtyp)
        #print(df_state.head())
        
update_census('exp', 'HS6', 'from+2021-10')

-
1010
1220
1610
2010
2050
2080
2110
2150
2190
2230
2250
2320
2360
2390
2410
2430
2440
2450
2470
2481
2482
2483
2484
2485
2486
2487
2488
2489
2720
2740
2774
2777
2779
2831
2839
3010
3070
3120
3150
3170
3310
3330
3350
3370
3510
3530
3550
3570
3720
4000
4010
4031
4039
4050
4091
4099
4120
4190
4210
4231
4239
4271
4272
4279
4280
4330
4351
4359
4370
4411
4419
4470
4490
4510
4550
4621
4622
4623
4631
4632
4633
4634
4635
4641
4642
4643
4644
4700
4710
4720
4730
4751
4752
4759
4791
4792
4793
4794
4801
4803
4804
4810
4840
4850
4870
4890
4910
5020
5040
5050
5070
5081
5082
5083
5110
5130
5170
5180
5200
5210
5230
5250
5310
5330
5350
5360
5380
5420
5460
5490
5520
5530
5550
5570
5590
5600
5601
5610
5650
5660
5682
5683
5700
5740
5790
Not able to get data 204
5800
5820
5830
5880
6021
6022
6023
6024
6029
6040
6141
6142
6143
6144
6150
6223
6224
6225
6226
6227
6412
6413
6414
6810
6820
6830
6862
6863
6864
7140
7210
7230
7250
7290
7321
7323
7380
7410
7420
7440
7450
7460
7470
7480
7490
7500
7510
7520
7530
754